**Name:** \_\_\_\_\_

**EID:** \_\_\_\_\_

# CS4487 - Tutorial 8: Non-Linear Dimensionality Reduction and Face Recognition

In this tutorial you will use non-linear dimensionality reduction on face images, and then train a classifier for face recognition. 

First we need to initialize Python.  Run the below cell.

In [ ]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
import joblib
from numpy import *
from sklearn import *
import glob
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
random.seed(100)
rbow = plt.get_cmap('rainbow')

## Loading Data and Pre-processing
We first need to load the images.  We will use the same dataset as Tutorial 7. Download `olivetti_py3.pkz` from Canvas, and place it in in the same directory as this ipynb file.  _DO NOT UNZIP IT_.  Then run the following cell to load the images.

In [ ]:
oli = datasets.fetch_olivetti_faces(data_home="./")
X = oli.data
Y = oli.target
img = oli.images
imgsize = oli.images[0].shape

Each image is a 64x64 array of pixel values, resulting in a 4096 dimensional vector.  Run the below code to show all the images!

In [ ]:
tmp = []
for i in range(0,400,20):
    tmp.append( hstack(img[i:i+20]) )
allimg = vstack(tmp)
plt.figure(figsize=(9,9))
plt.imshow(allimg, cmap='gray', interpolation='nearest')
plt.show()

Each person is considered as one class, and there are 10 images for each class.  In total there are 40 classes (people).  The data is already vectorized and put into the matrix `X`. Now we split the data into training and testing sets.

In [ ]:
# randomly split data into 80% train and 20% test set
trainX, testX, trainY, testY = \
  model_selection.train_test_split(X, Y, 
  train_size=0.80, test_size=0.20, random_state=4487)

print(trainX.shape)
print(testX.shape)

## Non-Linear Dimensionality Reduction - KPCA
The dimension of the data is too large (4096) so learning classifiers will take a long time.  Instead, our strategy is to use KPCA to reduce the dimension first and then use the KPCA weights as the representation for each image.  Run KPCA on the data using **10** principal components.  Use the RBF kernel with gamma=0.001.

In [ ]:
### INSERT YOUR CODE HERE
## HINT
# 1. decomposition.KernelPCA(n_components= , kernel= , gamma= , n_jobs= )

In [ ]:
### INSERT YOUR CODE HERE


The below function will plot the basis vectors of KPCA. Run the next 2 cells to view the PCs.  The kernel PCs are a combination of similarities to points in the training set.  The PCs are visualized by showing the top 5 positive and negative training examples, along with their coefficient $\alpha_i$.

In [ ]:
def plot_kbasis(model, imgsize, X):
    KK = model.n_components
    alphas = model.alphas_.T
    minmax = 5
    
    py = KK
    px = minmax*2
    for i in range(KK):
        # sort alphas
        inds = argsort(alphas[i,:])

        myi = r_[arange(-1,-minmax-1,-1), arange(minmax-1,-1,-1)]
        myinds = inds[myi]
        
        for j,jj in enumerate(myinds):
            plt.subplot(py,px,(j+1)+i*px)
            plt.imshow(X[jj,:].reshape(imgsize), interpolation='nearest')
            plt.gray()
            if alphas[i,jj]<0:
                mycol = 'b'
            else:
                mycol = 'r'
            plt.title("{:.3f}".format(alphas[i,jj]), fontsize=7, color=mycol)
            if (j==0):
                plt.ylabel('PC' + str(i+1))
            plt.xticks([])
            plt.yticks([])

In [ ]:
# run the function
plt.figure(figsize=(10,12))
plot_kbasis(kpca, imgsize, trainX)

_What is the interpretation for the KPCA basis?  What kind of faces do some of the PCs prefer?_
- **INSERT YOUR ANSWER HERE**

### Face Recognition
Now train a logistic classifier to do the face recognition.  Use the calculated KPCA representation as the new set of inputs.  Use cross-validation to set the hyperparameters of the classifier.  You do not need to do cross-validation for the number of components or kernel hyperparameters.  Calculate the average training and testing accuracies.  Remember to transform the test data into the KPCA representation too!


In [ ]:
### INSERT YOUR CODE HERE
## HINT 
# 1. linear_model.LogisticRegressionCV(Cs= , cv= , n_jobs= )
# 2. acc = metrics.accuracy_score( , )

In [ ]:
### INSERT YOUR CODE HERE

## Non-Linear Dimensionality Reduction - ICA

Next, we will use ICA to reduce the dimension.

- Applying the ICA with **20** principal components as the representation for each image. 
- Using the calculated ICA representation to  train a logistic classifier to do the face recognitio. 
- Calculating the average training and testing accuracies. 

In [ ]:
### INSERT YOUR CODE HERE
## HINT
# 1. decomposition.FastICA(n_components=  )
# 2. linear_model.LogisticRegressionCV(Cs= , cv= , n_jobs= )

In [ ]:
### INSERT YOUR CODE HERE


## Finding the best kernel and best number of components
Now try different kernels (poly, RBF), kernel parameters, and number of components to get the best test accuracy using KPCA.  
Train a logistic classifier for each one and see which dimension gives the best testing accuracy. 
Make a plots of number of components vs. test accuracy.

In [ ]:
## HINT
#1. KernelPCA(n_components=nc, kernel='rbf', gamma=, n_jobs=)
#2. LogisticRegressionCV(Cs=, cv=, n_jobs=)
#3. metrics.accuracy_score( , )

gammas = [0.0001, 0.0005, 0.001, 0.005, 0.01]
ncs = [5, 10, 15, 20, 25, 30]
nc = max(ncs)

trainacc = zeros((len(gammas), len(ncs)))
testacc  = zeros((len(gammas), len(ncs)))
### INSERT YOUR CODE HERE ###


In [ ]:
## HINT
#1. KernelPCA(n_components=nc, kernel='poly', degree=, n_jobs=)
#2. LogisticRegressionCV(Cs=, cv=, n_jobs=)
#3. metrics.accuracy_score( , )
degrees = [1,2,3,4,5]
trainacc_poly = zeros((len(degrees), len(ncs)))
testacc_poly  = zeros((len(degrees), len(ncs)))
### INSERT YOUR CODE HERE ###


In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
for i,gamma in enumerate(gammas):
    plt.plot(ncs, trainacc[i,:], '.-', color=rbow(float(i)/len(gammas)), label="RBF gamma="+str(gamma))
for i,d in enumerate(degrees):
    plt.plot(ncs, trainacc_poly[i,:], '.--', color=rbow(float(i)/len(degrees)), label="poly deg="+str(d))
plt.ylim(0,1.1)    

plt.legend(loc="lower right", fontsize=7)
plt.grid(True)
plt.subplot(1,2,2)
for i,gamma in enumerate(gammas):
    plt.plot(ncs, testacc[i,:], '.-', color=rbow(float(i)/len(gammas)))
for i,d in enumerate(degrees):
    plt.plot(ncs, testacc_poly[i,:], '.--', color=rbow(float(i)/len(degrees)))
plt.ylim(0,1.1)    
    
plt.grid(True)

_What is the best kernel and number of components?  View the prototypes for each compenent to see what they look like_
- **INSERT YOUR ANSWER HERE**

In [ ]:
### INSERT YOUR CODE HERE
## HINT
# 1. ii = unravel_index(argmax(testacc), testacc.shape)
# 2. gamma = gammas[ii[0]]
# 3. plot_kbasis(kpca, imgsize, trainX)